In [7]:
import pandas as pd
import os
import re
from ipysheet import from_dataframe, to_dataframe


In [11]:
# NFCU
path = "./in/transactions.CSV"
df = pd.read_csv(path)
df["Date"] = pd.to_datetime(df.Date)
df.drop("No.", axis=1, inplace=True)
df.rename(columns={"Description": "Payee"}, inplace=True)
df.to_csv("./out/nfcu.csv")


In [3]:
# Ally
path = "./in/ally.csv"
df = pd.read_csv(path)

df.drop(columns=[" Time", " Type"], inplace=True)
df.rename(columns={" Description": "Payee"}, inplace=True)

df.to_csv("./out/ally.csv")


In [4]:
# Chase
path = "./in/"
for filename in os.listdir(path):
    if re.match("Chase.*.CSV", filename):
        path = path + filename
df = pd.read_csv(path)
df.drop(columns=["Transaction Date", "Category", "Type"], inplace=True)
df.rename(columns={"Post Date": "Date"}, inplace=True)

df.to_csv("./out/Chase.csv")


In [6]:
# Amex
path = "./in/activity.csv"
df = pd.read_csv(path)

df.rename(columns={"Description": "Payee"}, inplace=True)

df.to_csv("./out/amex.csv")


In [12]:
# PayPal
path = "./in/recent-transactions.csv"
paypal_df = pd.read_csv(path)

paypal_df.rename(columns={"date": "Date", "amount": "Amount", "description": "Payee"}, inplace=True)
paypal_df["Date"] = pd.to_datetime(paypal_df.Date)
paypal_df.drop(columns=["id"], inplace=True)
paypal_df["Amount"] = (
    paypal_df["Amount"]
    .replace("[\$,)]", "", regex=True)
    .replace("[(]", "-", regex=True)
    .astype(float)
)
paypal_df.to_csv("./out/paypal.csv")

AttributeError: 'DataFrame' object has no attribute 'to_datetime'

In [8]:
# Full View
# creates csv for each instututionAccount from fullview transaction aggregation output
path = "./in/"
for filename in os.listdir(path):
    if re.match("transactions_.*.csv", filename):
        path = path + filename

df = pd.read_csv(path)

# fixes dates
df["Date"] = pd.to_datetime(df.Date)

# creates institution-account column and list of uniques
df["Acct"] = df["Institution"] + df["Account"]
accts = df["Acct"].unique().tolist()

# creates each list of transactions per account, only including posted transactions
for acct in accts:
    acct_df = df.loc[
        (df["Acct"] == acct)
        & (df["Is Hidden"] == "No")
        & (df["Is Pending"] == "No")
    ]

    # format for actual by dropping unused columns, renaming payee, and formatting amounts
    acct_df.drop(
        columns=["Acct", "Institution", "Account", "Is Hidden", "Category", "Is Pending"],
        inplace=True,
    )
    acct_df.rename(columns={"Description": "Payee"}, inplace=True)
    acct_df["Amount"] = (
        acct_df["Amount"]
        .replace("[\$,)]", "", regex=True)
        .replace("[(]", "-", regex=True)
        .astype(float)
    )

    acct_df.to_csv(f"./out/{acct.replace('*','')}.csv")


/var/folders/yg/8s5szk2j21z1wv_hn9s0gv0r0000gn/T/ipykernel_5579/2420949976.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acct_df.drop(
/var/folders/yg/8s5szk2j21z1wv_hn9s0gv0r0000gn/T/ipykernel_5579/2420949976.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acct_df.rename(columns={"Description": "Payee"}, inplace=True)
/var/folders/yg/8s5szk2j21z1wv_hn9s0gv0r0000gn/T/ipykernel_5579/2420949976.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or